In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
from core.backtesting import BacktestingEngine

backtesting = BacktestingEngine(root_path=root_path, load_cached_data=True)

2025-01-22 08:17:52,359 - root - ERROR - Error writing configs: [Errno 2] No such file or directory: '/Users/michael.zakharenkov/Projects/RolexVThres/.venv/lib/python3.10/site-packages/conf/conf_client.yml'
Traceback (most recent call last):
  File "/Users/michael.zakharenkov/Projects/RolexVThres/.venv/lib/python3.10/site-packages/hummingbot/client/config/config_helpers.py", line 876, in save_to_yml
    with open(yml_path, "w", encoding="utf-8") as outfile:
FileNotFoundError: [Errno 2] No such file or directory: '/Users/michael.zakharenkov/Projects/RolexVThres/.venv/lib/python3.10/site-packages/conf/conf_client.yml'


In [17]:
from hummingbot.strategy_v2.executors.position_executor.data_types import TrailingStop
from r1_controller import R1ControllerConfig
import datetime
from decimal import Decimal

# Controller configuration
connector_name = "binance_perpetual"
trading_pair = "ADA-USDT"
interval = "1h"
ema_short: int = 10
sma_long: int = 50
roc_length: int = 14
bbands_length: int = 20
bbands_std: int = 2
rsi_length: int = 14
adx_period: int = 14
spike_zscore: int = 2
total_amount_quote = 1000
max_executors_per_side = 1
take_profit = 0.1
stop_loss = 0.05
trailing_stop_activation_price = 0.007
trailing_stop_trailing_delta = 0.003
time_limit = 60 * 60 * 24 * 2
cooldown_time = 60 * 15

start = int(datetime.datetime(2025, 1, 1).timestamp())
end = int(datetime.datetime(2025, 2, 1).timestamp())


# Creating the instance of the configuration and the controller
config = R1ControllerConfig(
    connector_name=connector_name,
    trading_pair=trading_pair,
    interval=interval,
    ema_short=ema_short,
    sma_long=sma_long,
    roc_length=roc_length,
    bbands_length=bbands_length,
    bbands_std=bbands_std,
    rsi_length=rsi_length,
    adx_period=adx_period,
    spike_zscore=spike_zscore,
    total_amount_quote=Decimal(total_amount_quote),
    take_profit=Decimal(take_profit),
    stop_loss=Decimal(stop_loss),
    trailing_stop=TrailingStop(
        activation_price=Decimal(trailing_stop_activation_price), trailing_delta=Decimal(trailing_stop_trailing_delta)
    ),
    time_limit=time_limit,
    max_executors_per_side=max_executors_per_side,
    cooldown_time=cooldown_time,
)

In [18]:
backtesting_result = await backtesting.run_backtesting(config, start, end, interval)

2025-01-22 08:20:26,303 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x314b5f940>
2025-01-22 08:20:26,304 - asyncio - ERROR - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x314ba0a60>, 345053.91573325)])']
connector: <aiohttp.connector.TCPConnector object at 0x314b5eb60>
2025-01-22 08:20:27,631 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x314b5c220>
2025-01-22 08:20:27,631 - asyncio - ERROR - Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x314ba0520>, 345055.244227583)])']
connector: <aiohttp.connector.TCPConnector object at 0x314b5f2e0>


In [19]:
# Let's see what is inside the backtesting results
print(backtesting_result.get_results_summary())
fig = backtesting_result.get_backtesting_figure()

# Update layout for dark theme
fig.update_layout(
    # title=f'{exchange} - {trading_pair} - {timeframe}',
    width=1200,
    height=800,
    font=dict(color="#e1e1e1"),
    plot_bgcolor="#1e1e1e",
    paper_bgcolor="#1e1e1e",
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor="rgba(0,0,0,0)"),
    showlegend=False,
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)

# Show the plot
fig.show()


Net PNL: $1.49 (0.15%) | Max Drawdown: $-26.50 (-2.65%)
Total Volume ($): 28000.00 | Sharpe Ratio: 0.45 | Profit Factor: 1.05
Total Executors: 14 | Accuracy Long: 0.10 | Accuracy Short: 0.00
Close Types: Take Profit: 0 | Stop Loss: 13 | Time Limit: 0 |
             Trailing Stop: 1 | Early Stop: 0



In [20]:
# 2. The executors dataframe: this is the dataframe that contains the information of the orders that were executed
import pandas as pd

executors_df = backtesting_result.executors_df
executors_df.head()

,id,timestamp,type,close_timestamp,close_type,status,config,net_pnl_pct,net_pnl_quote,cum_fees_quote,filled_amount_quote,is_active,is_trading,custom_info,controller_id,side
0,A6cafoC8X3bWahoxahQPqf4sswgZAXoy4Gcf6dJRqYSN,1735873200,position_executor,1735873200,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': 'A6cafoC8X3bWahoxahQPqf4sswgZAXoy4Gcf6d...,-0.0005999999999999999474378786779027450393186...,-0.5999999999999999777955395074968691915273666...,0.59999999999999997779553950749686919152736663...,2000,False,False,"{'close_price': 1.0265, 'level_id': None, 'sid...",None,BUY
1,6DEkCQiDmN1vBf4y2tPHTxtB4ber4PtC58MCMJsyLX7i,1735884000,position_executor,1735884000,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': '6DEkCQiDmN1vBf4y2tPHTxtB4ber4PtC58MCMJ...,-0.0005999999999999999474378786779027450393186...,-0.5999999999999999777955395074968691915273666...,0.59999999999999997779553950749686919152736663...,2000,False,False,"{'close_price': 1.0395, 'level_id': None, 'sid...",None,BUY
2,BHby7s3p31erHRCioLVR89RTLaD3dctQ3UruENtMMEBN,1735891200,position_executor,1735909200,CloseType.TRAILING_STOP,RunnableStatus.TERMINATED,{'id': 'BHby7s3p31erHRCioLVR89RTLaD3dctQ3UruEN...,0.02918191949274666699420954785182402702048420...,29.1819194927466654121417377609759569168090820...,0.59999999999999997779553950749686919152736663...,2000,False,False,"{'close_price': 1.0719, 'level_id': None, 'sid...",None,BUY
3,AySsGje4P2abh38uaeD6kT2UHWRvYbapE8jLQW31DRff,1736578800,position_executor,1736589600,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': 'AySsGje4P2abh38uaeD6kT2UHWRvYbapE8jLQW...,-0.0204956294846706070511554997892744722776114...,-20.495629484670608633223309880122542381286621...,0.59999999999999997779553950749686919152736663...,2000.000000000000227373675443232059478759765625,False,False,"{'close_price': 0.9381, 'level_id': None, 'sid...",None,SELL
4,9wQsydsxypVLvX8F6cyBHBmE8jfQdU4cJd3hM31ebfzs,1736640000,position_executor,1736640000,CloseType.STOP_LOSS,RunnableStatus.TERMINATED,{'id': '9wQsydsxypVLvX8F6cyBHBmE8jfQdU4cJd3hM3...,-0.0005999999999999999474378786779027450393186...,-0.5999999999999999777955395074968691915273666...,0.59999999999999997779553950749686919152736663...,2000,False,False,"{'close_price': 1.0152, 'level_id': None, 'sid...",None,BUY


In [11]:
import plotly.express as px

# Create a new column for profitability
executors_df["profitable"] = executors_df["net_pnl_quote"] > 0

# Create the scatter plot
fig = px.scatter(
    executors_df,
    x="timestamp",
    y="net_pnl_quote",
    title="PNL per Trade",
    color="profitable",
    color_discrete_map={True: "green", False: "red"},
    labels={"timestamp": "Timestamp", "net_pnl_quote": "Net PNL (Quote)"},
    hover_data=["filled_amount_quote", "side"],
)

# Customize the layout
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Net PNL (Quote)",
    legend_title="Profitable",
    font=dict(size=12, color="white"),
    showlegend=False,
    plot_bgcolor="rgba(0,0,0,0.8)",  # Dark background
    paper_bgcolor="rgba(0,0,0,0.8)",  # Dark background for the entire plot area
    xaxis=dict(gridcolor="gray"),
    yaxis=dict(gridcolor="gray"),
)

# Add a horizontal line at y=0 to clearly separate profits and losses
fig.add_hline(y=0, line_dash="dash", line_color="lightgray")

# Show the plot
fig.show()

In [12]:
fig = px.histogram(executors_df, x="net_pnl_quote", title="PNL Distribution")
fig.show()